In [ ]:
import pandas as pd
import sqlite3

### Importing the csv file as a DataFrame

In [ ]:
df = pd.read_csv("c2_consumer_complaints.csv")
df.shape

In [ ]:
df.head()

### Creating a database

In [ ]:
db = sqlite3.connect("complaints.db")

# defining our helper function for running queries
def run_query(query):
    return pd.read_sql_query(query, db)

### Loading the data into the database

In [ ]:
df.to_sql(name="complaints", con=db, if_exists="append", index=False)

In [ ]:
# checking that all the data was loaded
run_query("SELECT COUNT(*) FROM complaints")

### Computing the number of complaints per submission method

In [ ]:
query = """
SELECT submitted_via AS 'Method', COUNT(*) AS 'Number'
FROM complaints
GROUP BY submitted_via
ORDER BY COUNT(*) DESC;
"""
run_query(query)

### Adding the total number of complaints

In [ ]:
query = """
SELECT submitted_via AS 'Method', COUNT(*) AS 'Number',
   (SELECT COUNT(*) FROM complaints) AS 'Total'
FROM complaints
GROUP BY submitted_via
ORDER BY COUNT(*) DESC;
"""
run_query(query)

### Computing the percentage of complaints of each submission method

In [ ]:
query = """

SELECT *, (CAST (q1.Number AS double precision)/CAST (q1.Total AS double precision))*100 AS 'Percent'
FROM (SELECT submitted_via AS 'Method', COUNT(*) AS 'Number',
   (SELECT COUNT(*) FROM complaints) AS 'Total'
FROM complaints
GROUP BY submitted_via
ORDER BY COUNT(*) DESC) q1;
"""
run_query(query)